In [1]:
import numpy as np
import pandas as pd

In [2]:
messages = pd.read_excel('../data/messages_updated.xlsx')
print(messages.shape)

(987, 10)


In [3]:
messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'].isna().sum()

130

In [4]:
# Удалим записи об авариях:

len1 = len(messages)
lst = []

for index, row in messages.iterrows():
    start = row['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ']
    end = row['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ']

    # с нулевой продолжительностью
    if start == end:
        lst.append(index)
        continue

    # с отрицательной продолжительностью, исключая строки с NaT
    if not pd.isnull(end) and start > end:
        lst.append(index)
        continue

    # M1 продолжительностью до 5 минут, исключая строки с NaT
    if not pd.isnull(end) and end - start <= pd.Timedelta(minutes=5):
            lst.append(index)

messages.drop(lst, inplace=True)

len2 = len(messages)
print('Удалили', len1 - len2, 'строк')

Удалили 17 строк


In [5]:
# Расчет минимального времени устранения неисправности

timedelta = messages[['ИМЯ_МАШИНЫ',
                      'НАЗВАНИЕ_ТЕХ_МЕСТА',
                      'ВИД_СООБЩЕНИЯ',
                      'ОПИСАНИЕ', 
                      'ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ',
                      'ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ',
                      'ТЕКСТ_ГРУППЫ_КОДОВ']].copy()

timedelta['ПРОДОЛЖИТЕЛЬНОСТЬ'] = \
    timedelta['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'] - timedelta['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ']

#timedelta.sort_values(by=['ПРОДОЛЖИТЕЛЬНОСТЬ'], ascending=True).to_excel('../data/timedelta.xlsx')

In [6]:
# Минимальная продолжительность M1

M1_min = timedelta[timedelta['ВИД_СООБЩЕНИЯ'] == 'M1']['ПРОДОЛЖИТЕЛЬНОСТЬ'].min()
timedelta[timedelta['ВИД_СООБЩЕНИЯ'] == 'M1'].sort_values(by=['ПРОДОЛЖИТЕЛЬНОСТЬ'], ascending=True).head()

,ИМЯ_МАШИНЫ,НАЗВАНИЕ_ТЕХ_МЕСТА,ВИД_СООБЩЕНИЯ,ОПИСАНИЕ,ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ,ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ,ТЕКСТ_ГРУППЫ_КОДОВ,ПРОДОЛЖИТЕЛЬНОСТЬ
986,ЭКСГАУСТЕР А/М №9,ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №9,M1,остановка эксг. №9 (2РУ МТЗ),2021-12-29 16:27:02,2021-12-29 16:33:03,ТЕХНИЧЕСКИЕ НЕПЛАНОВЫЕ,0 days 00:06:01
339,ЭКСГАУСТЕР А/М №7,ЭКСГАУСТЕР А/М №7,M1,остановка эксгаустера №7 (МТЗ),2020-01-10 19:38:18,2020-01-10 19:44:19,ТЕХНОЛОГИЧЕСКИЕ НЕПЛАНОВЫЕ,0 days 00:06:01
176,ЭКСГАУСТЕР А/М №4,ПОДШИПНИК ОПОРНЫЙ №2 ЭКСГ. №4,M1,остановка эксгаустера (температура),2019-07-30 19:14:34,2019-07-30 19:20:35,ТЕХНИЧЕСКИЕ НЕПЛАНОВЫЕ,0 days 00:06:01
892,ЭКСГАУСТЕР А/М №5,ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ1,M1,остановка эк-ра №5 (2РУ МТЗ),2021-09-10 23:18:08,2021-09-10 23:25:09,ТЕХНИЧЕСКИЕ НЕПЛАНОВЫЕ,0 days 00:07:01
692,ЭКСГАУСТЕР А/М №4,ЭЛЕКТРОАППАРАТУРА ЭКСГ. №4,M1,остановка экс-ра №4 (РТ газ. задвижки),2021-02-17 19:50:34,2021-02-17 19:57:35,ТЕХНИЧЕСКИЕ НЕПЛАНОВЫЕ,0 days 00:07:01


In [7]:
# Минимальная продолжительность M3

M3_min = timedelta[timedelta['ВИД_СООБЩЕНИЯ'] == 'M3']['ПРОДОЛЖИТЕЛЬНОСТЬ'].min()
timedelta[timedelta['ВИД_СООБЩЕНИЯ'] == 'M3'].sort_values(by=['ПРОДОЛЖИТЕЛЬНОСТЬ'], ascending=True).head()

,ИМЯ_МАШИНЫ,НАЗВАНИЕ_ТЕХ_МЕСТА,ВИД_СООБЩЕНИЯ,ОПИСАНИЕ,ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ,ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ,ТЕКСТ_ГРУППЫ_КОДОВ,ПРОДОЛЖИТЕЛЬНОСТЬ
935,ЭКСГАУСТЕР А/М №6,ТР-Р ТМ-4000-10/6 ЭКСГ. №6,M3,Нагрев контакта,2021-11-08 12:42:00,2021-11-08 12:50:00,NaN,0 days 00:08:00
947,ЭКСГАУСТЕР А/М №9,ТР-Р ТМ-6300-10/6 ЭКСГ. №9,M3,Подтеки масла на изоляторах,2021-11-23 15:35:00,2021-11-23 15:45:45,NaN,0 days 00:10:45
946,ЭКСГАУСТЕР А/М №9,ТР-Р ТМ-6300-10/6 ЭКСГ. №9,M3,Подгар контактов термосигнализатора,2021-11-23 15:33:00,2021-11-23 15:45:38,NaN,0 days 00:12:38
945,ЭКСГАУСТЕР А/М №9,ТР-Р ТМ-6300-10/6 ЭКСГ. №9,M3,Низкий уровень масла в тр-ре,2021-11-23 15:32:00,2021-11-23 15:45:32,NaN,0 days 00:13:32
944,ЭКСГАУСТЕР А/М №9,ТР-Р ТМ-6300-10/6 ЭКСГ. №9,M3,Термосигнализатор не закреплен,2021-11-23 15:31:00,2021-11-23 15:45:25,NaN,0 days 00:14:25


In [8]:
# Округление дат

messages['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ'] = messages['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ'].apply(lambda t: t.ceil('10S'))

messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'] = messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'].apply(
    lambda t: t.floor('10S') if not pd.isnull(t) else np.nan  # чтобы не округлять NaT
)
messages.head()

,Unnamed: 0,МАШИНА,ИМЯ_МАШИНЫ,ТЕХ_МЕСТО,НАЗВАНИЕ_ТЕХ_МЕСТА,ВИД_СООБЩЕНИЯ,ОПИСАНИЕ,ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ,ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ,ТЕКСТ_ГРУППЫ_КОДОВ
0,390,AA2/006-006,ЭКСГАУСТЕР А/М №9,AA2/006-006-002-008,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9,M3,неисправен двигатель,2019-01-21 00:00:00,2019-02-25 00:00:00,NaN
1,391,CH-AGP-AG2/011-005,ЭКСГАУСТЕР А/М №9,CH-AGP-AG2/011-005-002,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9,M3,неисправен двигатель,2019-01-21 12:26:10,2019-02-25 00:00:00,NaN
2,392,AA2/002-006,ЭКСГАУСТЕР А/М №5,AA2/002-006-002-004,МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ. №5,M3,менять прокладку на подачу воды в охлади,2019-01-21 15:35:30,2019-01-23 11:00:00,NaN
3,393,CH-AGP-AG2/003-005,ЭКСГАУСТЕР А/М №5,CH-AGP-AG2/003-005-002-004,МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ. №5,M3,менять прокладку на подачу воды в охлади,2019-01-21 16:33:00,2019-01-22 00:00:00,NaN
4,394,CH-AGP-AG2/003-005,ЭКСГАУСТЕР А/М №5,CH-AGP-AG2/003-005-002-004,МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ. №5,M3,менять прокладку на подачу воды в охлади,2019-01-21 16:36:10,2019-06-04 15:35:20,NaN


In [9]:
messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'].isna().sum()

130

In [10]:
# Сохранение для дальнейшей работы
messages.to_excel('../data/processed/messages_processed.xlsx', index=False)